In [1]:
%defaultDatasource jdbc:h2:mem:db

# Primeiro conjunto de tabelas para consultas básicas

In [2]:
DROP TABLE IF EXISTS Taxi;
DROP TABLE IF EXISTS Cliente;
DROP TABLE IF EXISTS Corrida;

CREATE TABLE Taxi (
  Placa VARCHAR(7) NOT NULL,
  Marca VARCHAR(30) NOT NULL,
  Modelo VARCHAR(30) NOT NULL,
  AnoFab INTEGER,
  Licenca VARCHAR(9),
  PRIMARY KEY(Placa)
);

CREATE TABLE Cliente (
  CliId VARCHAR(4) NOT NULL,
  Nome VARCHAR(80) NOT NULL,
  CPF VARCHAR(14) NOT NULL,
  PRIMARY KEY(CliId)
);

CREATE TABLE Corrida (
  CliId VARCHAR(4) NOT NULL,
  Placa VARCHAR(7) NOT NULL,
  DataPedido DATE NOT NULL,
  PRIMARY KEY(CliId, Placa, DataPedido),
  FOREIGN KEY(CliId)
    REFERENCES Cliente(CliId)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(Placa)
    REFERENCES Taxi(Placa)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
);

In [3]:
INSERT INTO Cliente VALUES ('1532', 'Asdrúbal', '448.754.253-65');
INSERT INTO Cliente VALUES ('1755', 'Doriana', '567.387.387-44');
INSERT INTO Cliente VALUES ('1780', 'Quincas', '546.373.762-02');

INSERT INTO Taxi VALUES ('DAE6534', 'Ford', 'Fiesta', 1999, 'MN572345');
INSERT INTO Taxi VALUES ('DKL4598', 'Wolkswagen', 'Gol', 2001, 'AU876543');
INSERT INTO Taxi VALUES ('DKL7878', 'Ford', 'Fiesta', 2001, 'OP102938');
INSERT INTO Taxi VALUES ('JDM8776', 'Wolkswagen', 'Santana', 2002, 'QM365923');
INSERT INTO Taxi VALUES ('JJM3692', 'Chevrolet', 'Corsa', 1999, 'UU335577');

INSERT INTO Corrida VALUES ('1755', 'DAE6534', '2003-02-15');
INSERT INTO Corrida VALUES ('1780', 'JDM8776', '2003-02-18');
INSERT INTO Corrida VALUES ('1755', 'DKL7878', '2003-02-16');
INSERT INTO Corrida VALUES ('1780', 'DKL4598', '2003-02-17');
INSERT INTO Corrida VALUES ('1532', 'DKL4598', '2003-02-18');
INSERT INTO Corrida VALUES ('1780', 'DAE6534', '2003-02-16');

# DELETE

### Excluindo as corridas feitas pelo Táxi de placa `DAE6534`

Tabela de Corridas antes:

In [4]:
SELECT * FROM Corrida;

Excluindo:

In [5]:
DELETE FROM Corrida
       WHERE Placa = 'DAE6534';

Tabela depois:

In [6]:
SELECT * FROM Corrida;

## Violação de Integridade

### Excluindo um Táxi que possui corridas associadas

Este exemplo irá gerar um erro de violação de chave estrangeira.

Tabela de Táxis:

In [7]:
SELECT * FROM Taxi;

Corridas para o Taxi de placa `DKL4598`:

In [8]:
SELECT * FROM Corrida WHERE Placa = 'DKL4598';

Tentativa de excluir o Táxi da tabela:

In [9]:
DELETE FROM Taxi WHERE Placa = 'DKL4598';

org.h2.jdbc.JdbcSQLException:  Referential integrity constraint violation

Excluindo as corridas desse Taxi:

In [10]:
DELETE FROM Corrida WHERE Placa = 'DKL4598';

Agora a exclusão funciona:

In [11]:
DELETE FROM Taxi WHERE Placa = 'DKL4598';

Tabela sem o Taxi:

In [12]:
SELECT * FROM Taxi;

# UPDATE

### Alterando o ano de fabricação (para 2002) do Táxi que possui placa `DKL7878`

In [13]:
UPDATE Taxi
       SET AnoFab = 2002
       WHERE Placa = 'DKL7878';

In [14]:
SELECT * FROM Taxi;

Expressões de atualização:

In [15]:
UPDATE Taxi
       SET AnoFab = AnoFab + 1
       WHERE Placa = 'DKL7878';

In [16]:
SELECT * FROM Taxi;

## UPDATE em mais de uma Tabela

### Alterando a Placa de um Taxi

Tabela de Corridas:

In [17]:
SELECT * FROM Corrida;

Tentativa de alterar a placa do Táxi `JDM8776` que possui uma corrida associada (violação de integridade):

In [18]:
UPDATE Taxi
       SET Placa = 'JDM8700'
       WHERE Placa = 'JDM8776';

org.h2.jdbc.JdbcSQLException:  Referential integrity constraint violation

Alterando a cláusula da tabela `Corrida` para `CASCADE`:

In [19]:
DROP TABLE Corrida;

CREATE TABLE Corrida (
  CliId VARCHAR(4) NOT NULL,
  Placa VARCHAR(7) NOT NULL,
  DataPedido DATE NOT NULL,
  PRIMARY KEY(CliId, Placa, DataPedido),
  FOREIGN KEY(CliId)
    REFERENCES Cliente(CliId)
      ON DELETE NO ACTION
      ON UPDATE CASCADE,
  FOREIGN KEY(Placa)
    REFERENCES Taxi(Placa)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

In [20]:
INSERT INTO Corrida VALUES ('1755', 'DKL7878', '2003-02-16');
INSERT INTO Corrida VALUES ('1780', 'JDM8776', '2003-02-18');

Agora a atualização funciona em cascata, ou seja, a placa é alterada tanto na tabela `Taxi` quanto na tabela `Corrida`:

In [21]:
UPDATE Taxi
       SET Placa = 'JDM8700'
       WHERE Placa = 'JDM8776';

Tabelas modificadas:

In [22]:
SELECT * FROM Taxi;
SELECT * FROM Corrida;

# Segundo conjunto de tabelas para consultas avançadas

In [23]:
DROP TABLE IF EXISTS Motorista;
DROP TABLE IF EXISTS Zona;
DROP TABLE IF EXISTS Fila;

CREATE TABLE Motorista (
  CNH VARCHAR(6) NOT NULL,
  Nome VARCHAR(80) NOT NULL,
  Ativo BOOLEAN,
  Placa VARCHAR(7) NOT NULL,
  PRIMARY KEY(CNH),
  FOREIGN KEY(Placa)
    REFERENCES Taxi(Placa)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
);

CREATE TABLE Zona (
  Zona VARCHAR(40) NOT NULL,
  PRIMARY KEY(Zona)
);

CREATE TABLE Fila (
   Zona VARCHAR(40) NOT NULL,
   CNH VARCHAR(6) NOT NULL,
   DataHoraIn TIMESTAMP,
   DataHoraOut TIMESTAMP,
   KmIn INTEGER,
   PRIMARY KEY (Zona, CNH),
   FOREIGN KEY(Zona)
     REFERENCES Zona(Zona)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION,
   FOREIGN KEY(CNH)
     REFERENCES Motorista(CNH)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION
);

In [24]:
INSERT INTO Taxi VALUES ('DXF5263', 'Ford', 'Escort', 2001, 'GG5263526');
INSERT INTO Taxi VALUES ('MJN6578', 'Wolkswagen', 'Santana', 1998, 'KL856326');
INSERT INTO Taxi VALUES ('MLA4545', 'Wolkswagen', 'Gol', 2000, 'YK6574837');
INSERT INTO Taxi VALUES ('DXA7878', 'Ford', 'Escort', 2000, 'GG5263526');
INSERT INTO Taxi VALUES ('DXF6767', 'Wolkswagen', 'Santana', 1998, 'YK7890081');
INSERT INTO Taxi VALUES ('DXF6868', 'Wolkswagen', 'Santana', 1998, 'YK7890082');
INSERT INTO Taxi VALUES ('DXB7070', 'Ford', 'Fiesta', 2015, 'GG101112');

INSERT INTO Motorista VALUES ('657483', 'Asdrubal', TRUE, 'DXF5263');
INSERT INTO Motorista VALUES ('567892', 'Quincas', TRUE, 'MLA4545');
INSERT INTO Motorista VALUES ('452635', 'Zandor', TRUE, 'DXA7878');
INSERT INTO Motorista VALUES ('452452', 'Alcebiades', TRUE, 'DXF6767');
INSERT INTO Motorista VALUES ('555555', 'Bonerges', TRUE, 'DAE6534');
INSERT INTO Motorista VALUES ('987543', 'Horacio', TRUE, 'DXB7070');
INSERT INTO Motorista VALUES ('987654', 'Doriana', TRUE, 'JDM8700');
INSERT INTO Motorista VALUES ('389204', 'Melissa', TRUE, 'DXF6868');

INSERT INTO Zona VALUES ('Barão Geraldo');
INSERT INTO Zona VALUES ('Cambuí');
INSERT INTO Zona VALUES ('Taquaral');
INSERT INTO Zona VALUES ('Unicamp');

INSERT INTO Fila VALUES ('Barão Geraldo', '567892', '2002-06-05 09:00:00', '2002-06-05 09:30:00', 4630);
INSERT INTO Fila VALUES ('Barão Geraldo', '657483', '2002-06-05 07:30:00', '2002-06-05 07:45:00', 1567);
INSERT INTO Fila VALUES ('Taquaral', '452452', '2002-06-05 09:00:00', '2002-06-05 09:50:00', 5000);
INSERT INTO Fila VALUES ('Taquaral', '657483', '2002-06-06 08:00:00', '2002-06-06 08:07:00', 7900);
INSERT INTO Fila VALUES ('Unicamp', '452635', '2002-06-02 08:00:00', '2002-06-02 08:00:00', 4800);
INSERT INTO Fila VALUES ('Unicamp', '567892', '2002-06-06 06:00:00', '2002-06-06 06:00:00', 5263);
INSERT INTO Fila VALUES ('Unicamp', '657483', '2002-06-06 23:00:00', '2002-06-06 23:00:00', 4541);
INSERT INTO Fila VALUES ('Unicamp', '452452', '2002-06-05 10:30:00', '2002-06-05 10:30:00', 7800);
INSERT INTO Fila VALUES ('Taquaral', '555555', '2002-06-06 08:10:00', '2002-06-06 08:10:00', 7910);
INSERT INTO Fila VALUES ('Unicamp', '987654', '2002-06-05 10:30:00', '2002-06-05 10:35:00', 7850);

# Aninhamento na Atualização (`UPDATE/SELECT`)

O SELECT aninhado pode ser parte de uma operação de UPDATE.

Observe a relação de todos os motoristas. Note que todos os motoristas têm CNH válido.

In [25]:
SELECT CNH, Nome, Ativo
       FROM Motorista;

Considere que queremos considerar ativos apenas aqueles motoristas que estão na Fila.

Relação de Motoristas na Fila:

In [26]:
SELECT DISTINCT CNH
       FROM Fila;

A seguinte consulta retorna aqueles motoristas que não estão em nenhuma fila:

In [27]:
SELECT M.CNH, M.Nome
       FROM Motorista M
       WHERE M.CNH NOT IN (
           SELECT DISTINCT F.CNH
           FROM Fila F)

Usando o UPDATE com SELECT aninhado, é possível tornar não ativos todos os motoristas que não aparecem em nenhuma fila:

In [28]:
UPDATE Motorista M
       SET M.Ativo = FALSE
       WHERE M.CNH NOT IN (
           SELECT DISTINCT F.CNH
           FROM Fila F);

Resultado após o UPDATE:

In [29]:
SELECT CNH, Nome, Ativo
       FROM Motorista;

# Aninhamento na Exclusão (`DELETE/SELECT`)

O SELECT aninhado pode ser parte de uma operação de UPDATE.

Dando continuidade à questão anterior, em vez de tornar inativos os Motoristas que não aparecem em nenhuma Fila, podemos decidir excluí-los:

In [30]:
DELETE FROM Motorista M
       WHERE M.CNH NOT IN (
           SELECT DISTINCT F.CNH
           FROM Fila F);

Resultado após o DELETE:

In [31]:
SELECT CNH, Ativo
       FROM Motorista;